In [3]:
!pip install opencv-python scikit-learn

In [41]:
!pip install Pillow

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
import random 
from collections import defaultdict

In [2]:
try:
    from PIL import Image
except ImportError:
    import sys
    !{sys.executable} -m pip install pillow
    from PIL import Image

In [4]:
#dataset path 
base_path = r'C:\Users\Chhama\OneDrive\Desktop\Banana'

In [5]:
#resize all images 
IMG_SIZE = (224, 224)

In [6]:
# Balancing and loading function
def load_and_balance_dataset(folder_path, target_count=None):
    X, y = [], []
    class_data = {}

    # First pass to collect and resize images
    for folder in os.listdir(folder_path):
        if not folder.startswith('Banana'):
            continue
        try:
            range_part = folder.split('(')[1].split(')')[0]
            low, high = map(int, range_part.split('-'))
            avg_shelf_life = (low + high) / 2.0
        except:
            continue

        folder_full_path = os.path.join(folder_path, folder)
        class_images = []

        for img_file in os.listdir(folder_full_path):
            img_path = os.path.join(folder_full_path, img_file)
            img = cv2.imread(img_path)

            if img is not None:
                img = cv2.resize(img, IMG_SIZE)
                img = img / 255.0
                class_images.append(img)

        class_data[folder] = (class_images, avg_shelf_life)

    # Determine max class size
    if not target_count:
        target_count = max(len(images) for images, _ in class_data.values())

    augmentor = ImageDataGenerator(
        rotation_range=15,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
        # brightness_range disabled
    )

    # Apply balancing with augmentation
    for folder, (images, shelf_life) in class_data.items():
        current_count = len(images)
        if current_count < target_count:
            to_add = target_count - current_count
            print(f"Augmenting class '{folder}' with {to_add} images...")

            for _ in range(to_add):
                img = random.choice(images)
                img_aug = augmentor.random_transform(img)
                X.append(img_aug)
                y.append(shelf_life)

        # Add original images
        X.extend(images)
        y.extend([shelf_life] * len(images))

    print("Final class distribution after balancing:")
    for folder, (images, _) in class_data.items():
        print(f"{folder}: {target_count}")

    return np.array(X), np.array(y)

In [7]:
# Load and balance dataset
X, y = load_and_balance_dataset(base_path)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Augmenting class 'Banana(1-5)' with 102 images...
Augmenting class 'Banana(10-15)' with 100 images...
Augmenting class 'Banana(5-10)' with 69 images...
Final class distribution after balancing:
Banana(1-5): 180
Banana(10-15): 180
Banana(15-20): 180
Banana(5-10): 180


In [8]:
# Build the ResNet50-based regression model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dense(1, activation='linear')(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=1e-4), loss=tf.keras.losses.Huber(), metrics=['mae'])

# Callbacks
checkpoint_path = "best_banana_model.h5"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)
]

In [81]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=40,
    batch_size=32,
    callbacks=callbacks
)

Epoch 1/40
18/18 [==============================] - 95s 5s/step - loss: 4.2088 - mae: 4.6740 - val_loss: 10.1171 - val_mae: 10.6171 - lr: 1.0000e-04
Epoch 2/40
18/18 [==============================] - 77s 4s/step - loss: 2.1495 - mae: 2.6111 - val_loss: 9.8516 - val_mae: 10.3516 - lr: 1.0000e-04
Epoch 3/40
18/18 [==============================] - 77s 4s/step - loss: 1.7733 - mae: 2.2243 - val_loss: 9.7198 - val_mae: 10.2198 - lr: 1.0000e-04
Epoch 4/40
18/18 [==============================] - 74s 4s/step - loss: 1.6526 - mae: 2.1060 - val_loss: 9.4880 - val_mae: 9.9880 - lr: 1.0000e-04
Epoch 5/40
18/18 [==============================] - 73s 4s/step - loss: 1.5101 - mae: 1.9459 - val_loss: 9.2409 - val_mae: 9.7409 - lr: 1.0000e-04
Epoch 6/40
18/18 [==============================] - 74s 4s/step - loss: 1.6498 - mae: 2.1033 - val_loss: 8.8420 - val_mae: 9.3420 - lr: 1.0000e-04
Epoch 7/40
18/18 [==============================] - 72s 4s/step - loss: 1.3031 - mae: 1.7409 - val_loss: 8.4869 - 

In [82]:
#Evaluate Final Model on Validation Set
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss:.4f}, Validation MAE: {val_mae:.4f} days") 

5/5 [==============================] - 13s 3s/step - loss: 0.6768 - mae: 0.9999
Validation Loss: 0.6768, Validation MAE: 0.9999 days


In [9]:
# Load the best model 
model.load_weights("best_banana_model.h5")

In [10]:
# Testing on a sample image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

img_path = r'C:\Users\Chhama\OneDrive\Desktop\Banana\Banana(5-10)\frame270.jpg'
img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

predicted_shelf_life = model.predict(img_array)[0][0]
print(f"\nPredicted Shelf Life: {predicted_shelf_life:.2f} days")

# Sustainable distribution logic
def shelf_life_action(days):
    if days > 10:
        return "Store in warehouse or export to far locations"
    elif days > 5:
        return "Distribute to fruit vendors or nearby markets"
    else:
        return "Send to processing units for juices or quick-sale outlets"

action = shelf_life_action(predicted_shelf_life)
print(f"Recommended Sustainability Action: {action}")

1/1 [==============================] - 3s 3s/step

Predicted Shelf Life: 8.01 days
Recommended Sustainability Action: Distribute to fruit vendors or nearby markets
